In [1]:
import pandas as pd

In [2]:
folder_path = r"C:\Users\rodge\OneDrive\Desktop\SML\Group Project\comp90051\data\rawData\unzip_file"
patient_info = pd.read_csv(folder_path + r"\patient_information.csv")
post_op = pd.read_csv(folder_path + r"\patient_post_op_complications.csv")

In [3]:
post_op_count = post_op['Element_abbr'].value_counts()
post_op_count

Element_abbr
AN Post-op Complications    200139
Other                         1094
Cardiovascular                 866
Respiratory                    748
Airway                         375
Metabolic                      154
Neurological                   147
Administrative                 118
Injury/Infection               117
Medication                      94
Regional                        61
Chronic Pain                    32
Name: count, dtype: int64

In [4]:
Cardiovascular = post_op[post_op['Element_abbr'] == 'Cardiovascular']
Cardiovascular

,LOG_ID,MRN,Element_Name,CONTEXT_NAME,Element_abbr,SMRTDTA_ELEM_VALUE
181,0ecf38ed306338f6,4c29aee08612d793,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Hypotension (SBP<80 for 10 min)
195,595439a55deda0d5,71d77695a69b26d2,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Hypotension (SBP<80 for 10 min)
472,f8e8289e5b983a10,f523e63fb030f0fa,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Instability (hemodynamic)
487,99d83372f55a8aeb,f523e63fb030f0fa,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Arrest (cardiac)
557,3bf060182acba73f,f523e63fb030f0fa,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Ischemia (myocardial)
...,...,...,...,...,...,...
203117,7dde9622ebc7a58f,b870a5296cc7b142,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Arrhythmia
203353,a7f49cbfafc390bc,e1cedd18cc73fd75,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Arrhythmia
203563,c88ad449b5d46ace,df69bd758e20ee84,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Arrhythmia
203680,0fab522159f4c69e,d7946ede59551209,AN POST-OP COMPLICATION CARDIOVASCULAR,ENCOUNTER,Cardiovascular,Ischemia (myocardial)


In [5]:
patient_info.head()

,LOG_ID,MRN,DISCH_DISP_C,DISCH_DISP,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,LOS,ICU_ADMIN_FLAG,SURGERY_DATE,BIRTH_DATE,...,PRIMARY_ANES_TYPE_NM,ASA_RATING_C,ASA_RATING,PATIENT_CLASS_GROUP,PATIENT_CLASS_NM,PRIMARY_PROCEDURE_NM,IN_OR_DTTM,OUT_OR_DTTM,AN_START_DATETIME,AN_STOP_DATETIME
0,cd1636c6279d73a0,2e92522a5f2a2a22,15.0,Home Routine,12/20/18 11:20,12/21/18 12:15,1.0,No,12/20/18 0:00,47,...,General,3.0,Severe Systemic Disease,Outpatient,Hospital Outpatient Surgery,"INSERTION, PUBOVAGINAL SLING, WITH CYSTOSCOPY",12/20/18 12:27,12/20/18 17:25,12/20/18 12:27,12/20/18 17:34
1,fc53c06ee2cfe438,b7a91e623d957d8f,16.0,Hospice Facility,9/27/19 9:02,10/18/19 18:13,21.0,Yes,9/27/19 0:00,81,...,General,3.0,Severe Systemic Disease,Outpatient,Hospital Outpatient Surgery,GI EGD DILATION,9/27/19 9:52,9/27/19 11:27,9/27/19 9:52,9/27/19 11:44
2,2f0090c47da34608,b68d6c7198f30f73,15.0,Home Routine,4/8/19 9:33,4/11/19 12:34,3.0,Yes,4/8/19 0:00,49,...,General,2.0,Mild Systemic Disease,Inpatient,Hospital Inpatient Surgery,CRANIECTOMY,4/8/19 14:41,4/8/19 18:46,4/8/19 14:41,4/8/19 18:53
3,6d3b621880c6ced4,46aa48e89b9c50e8,15.0,Home Routine,12/3/18 5:57,12/4/18 12:57,1.0,No,12/3/18 0:00,54,...,General,2.0,Mild Systemic Disease,Outpatient,Hospital Outpatient Surgery,"REVISION, RECONSTRUCTION, BREAST",12/3/18 8:09,12/3/18 11:49,12/3/18 8:09,12/3/18 11:52
4,82f89e10f2855420,c2756216cc89b795,15.0,Home Routine,4/19/19 6:19,4/20/19 9:26,1.0,No,4/19/19 0:00,44,...,General,2.0,Mild Systemic Disease,Outpatient,Hospital Outpatient Surgery,"REVISION, RECONSTRUCTION, BREAST",4/19/19 8:13,4/19/19 14:51,4/19/19 8:14,4/19/19 15:00
